In [169]:
from trackml.dataset import load_event
from trackml.dataset import load_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random

def proc_arr(x):
    for i in range(len(x) - 1):
        block_stat.ix[x[i], x[i+1]] += 1
        
def proc_arr_next_hit(x):
    for i in range(len(x) - 1):
        next_hits.ix[x[i], 'next_hit'] = x[i+1]

In [170]:
cnt = 0
for event_id, hits, cells, particles, truth in load_dataset('data/train_5.zip'):
    truth_hits = pd.concat([hits, truth.drop(['hit_id'], axis=1)], axis=1)
    truth_hits['dist'] = truth_hits.apply(lambda x: x[1]**2 + x[2]**2 + x[3]**2, axis=1)
    truth_hits['rad_dist'] = truth_hits.apply(lambda x: x[1]**2 + x[2]**2, axis=1)
    
    truth_hits = truth_hits[truth_hits.particle_id != 0].sort_values(['particle_id', 'rad_dist'])
    truth_hits['block'] = truth_hits['volume_id'].astype(str) + "_" + truth_hits['layer_id'].astype(str)
    if cnt == 1:
        break
    cnt += 1

In [210]:
cell_stats = cells.groupby('hit_id')['value'].mean().reset_index().rename({'value':'value_mean'},axis=1)

tmp = cells.groupby('hit_id')['value'].max().reset_index().rename({'value':'value_max'},axis=1)
cell_stats = pd.merge(cell_stats, tmp, on='hit_id', how='left')

tmp = cells.groupby('hit_id')['value'].min().reset_index().rename({'value':'value_min'},axis=1)
cell_stats = pd.merge(cell_stats, tmp, on='hit_id', how='left')

tmp = cells.groupby('hit_id')['value'].sum().reset_index().rename({'value':'value_sum'},axis=1)
cell_stats = pd.merge(cell_stats, tmp, on='hit_id', how='left')

tmp = cells.groupby('hit_id')['value'].std().reset_index().rename({'value':'value_std'},axis=1)
cell_stats = pd.merge(cell_stats, tmp, on='hit_id', how='left')

tmp = cells.groupby('hit_id')['value'].count().reset_index().rename({'value':'value_count'},axis=1)
cell_stats = pd.merge(cell_stats, tmp, on='hit_id', how='left')

In [225]:
cell_cols = list(cell_stats.columns.values[1:])

In [218]:
truth_hits = pd.merge(truth_hits, cell_stats, on='hit_id', how='left')

In [151]:
blocks = truth_hits.groupby('block')['particle_id'].count().reset_index()[['block']].values.reshape(48)
block_stat = pd.DataFrame(np.zeros((48, 48)))
block_stat.index = blocks
block_stat.columns = blocks

tmp = truth_hits.groupby('particle_id')['block'].apply(lambda x: x.tolist()).reset_index()
tmp['block'].apply(lambda x: proc_arr(x))
1

In [219]:
next_hits = pd.DataFrame(truth_hits['hit_id'].unique()).rename({0:'hit_id'},axis=1)
next_hits['next_hit'] = np.nan

truth_hits['next_particle'] = list(truth_hits['particle_id'].values[1:]) + [-1]
truth_hits['next_hit'] = list(truth_hits['hit_id'].values[1:]) + [-1]
truth_hits['not_end_hit'] = truth_hits['particle_id'] == truth_hits['next_particle']
truth_hits['next_hit']  = truth_hits['not_end_hit'] * truth_hits['next_hit']
del truth_hits['next_particle']


In [220]:
truth_hits.columns

Index(['hit_id', 'x', 'y', 'z', 'volume_id', 'layer_id', 'module_id',
       'particle_id', 'tx', 'ty', 'tz', 'tpx', 'tpy', 'tpz', 'weight', 'dist',
       'rad_dist', 'block', 'next_hit', 'not_end_hit', 'value_mean',
       'value_max', 'value_min', 'value_sum', 'value_std', 'value_count'],
      dtype='object')

In [226]:
geom_cols = ['x', 'y', 'z', 'volume_id', 'layer_id', 'module_id', 'dist', 'rad_dist', 'block']
X = truth_hits[geom_cols + cell_cols + ['next_hit']]
#X.columns = X.columns + '_1'
tmp = truth_hits[['hit_id'] + geom_cols + cell_cols]
#tmp.columns = tmp.columns + '_2'
X = pd.merge(X.rename({'next_hit':'hit_id'},axis=1), tmp, on=['hit_id'], how='left')
del X['hit_id']

In [227]:
X.shape

(82782, 30)

In [229]:
X.columns

Index(['x_x', 'y_x', 'z_x', 'volume_id_x', 'layer_id_x', 'module_id_x',
       'dist_x', 'rad_dist_x', 'block_x', 'value_mean_x', 'value_max_x',
       'value_min_x', 'value_sum_x', 'value_std_x', 'value_count_x', 'x_y',
       'y_y', 'z_y', 'volume_id_y', 'layer_id_y', 'module_id_y', 'dist_y',
       'rad_dist_y', 'block_y', 'value_mean_y', 'value_max_y', 'value_min_y',
       'value_sum_y', 'value_std_y', 'value_count_y'],
      dtype='object')

In [251]:
block_stat.ix['13_2']

12_10      28.0
12_12     450.0
12_2        0.0
12_4        0.0
12_6        4.0
12_8        2.0
13_2     1900.0
13_4     7044.0
13_6       44.0
13_8        0.0
14_10       4.0
14_12       0.0
14_2      520.0
14_4       16.0
14_6        8.0
14_8        0.0
16_10       0.0
16_12       0.0
16_2        0.0
16_4        0.0
16_6        0.0
16_8        0.0
17_2        0.0
17_4        0.0
18_10       0.0
18_12       0.0
18_2        0.0
18_4        0.0
18_6        0.0
18_8        0.0
7_10        0.0
7_12        0.0
7_14        0.0
7_2         0.0
7_4         0.0
7_6         0.0
7_8         0.0
8_2         0.0
8_4         0.0
8_6         0.0
8_8         0.0
9_10        0.0
9_12        0.0
9_14        0.0
9_2         0.0
9_4         0.0
9_6         0.0
9_8         0.0
Name: 13_2, dtype: float64

In [230]:
tmp = truth_hits[['hit_id'] + geom_cols + cell_cols]
tmp1 = truth_hits[['hit_id'] + geom_cols + cell_cols]
tmp = pd.merge(tmp, tmp1, on=['volume_id', 'layer_id', 'module_id'], how='outer')
tmp = tmp[tmp['hit_id_x'] != tmp['hit_id_y']]
del tmp['hit_id_x']
del tmp['hit_id_y']
tmp['volume_id_x'] = tmp['volume_id']
tmp['layer_id_x'] = tmp['layer_id']
tmp['module_id_x'] = tmp['module_id']

tmp['volume_id_y'] = tmp['volume_id']
tmp['layer_id_y'] = tmp['layer_id']
tmp['module_id_y'] = tmp['module_id']

del tmp['volume_id']
del tmp['layer_id']
del tmp['module_id']

tmp = tmp[X.columns]

In [232]:
tmp_idx = random.sample(list(tmp.index.values), 82782)
tmp = tmp.ix[tmp_idx]

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [233]:
tmp.shape

(82782, 30)

In [176]:
tmp

,x_x,y_x,z_x,volume_id_x,layer_id_x,module_id_x,dist_x,rad_dist_x,block_x,x_y,y_y,z_y,volume_id_y,layer_id_y,module_id_y,dist_y,rad_dist_y,block_y
1,13.244800,-32.379101,598.000000,9,2,36,3.588278e+05,1.223831e+03,9_2,41.012199,-53.863602,598.000000,9,2,36,3.621873e+05,4.583288e+03,9_2
2,13.244800,-32.379101,598.000000,9,2,36,3.588278e+05,1.223831e+03,9_2,27.431700,-46.247700,598.000000,9,2,36,3.604953e+05,2.891348e+03,9_2
3,13.244800,-32.379101,598.000000,9,2,36,3.588278e+05,1.223831e+03,9_2,45.788101,-63.762402,598.000000,9,2,36,3.637662e+05,6.162194e+03,9_2
4,13.244800,-32.379101,598.000000,9,2,36,3.588278e+05,1.223831e+03,9_2,51.560001,-77.971703,598.000000,9,2,36,3.663420e+05,8.738020e+03,9_2
5,13.244800,-32.379101,598.000000,9,2,36,3.588278e+05,1.223831e+03,9_2,16.193800,-30.110100,598.000000,9,2,36,3.587729e+05,1.168857e+03,9_2
6,13.244800,-32.379101,598.000000,9,2,36,3.588278e+05,1.223831e+03,9_2,11.028200,-31.824200,598.000000,9,2,36,3.587384e+05,1.134401e+03,9_2
7,13.244800,-32.379101,598.000000,9,2,36,3.588278e+05,1.223831e+03,9_2,24.465099,-37.004799,598.000000,9,2,36,3.595719e+05,1.967896e+03,9_2
8,13.244800,-32.379101,598.000000,9,2,36,3.588278e+05,1.223831e+03,9_2,9.373340,-30.463400,598.000000,9,2,36,3.586199e+05,1.015878e+03,9_2
9,13.244800,-32.379101,598.000000,9,2,36,3.588278e+05,1.223831e+03,9_2,21.290899,-24.743601,598.000000,9,2,36,3.586695e+05,1.065548e+03,9_2
10,13.244800,-32.379101,598.000000,9,2,36,3.588278e+05,1.223831e+03,9_2,34.333599,-51.386101,598.000000,9,2,36,3.614233e+05,3.819327e+03,9_2


In [234]:
X['target'] = 1
tmp['target'] = 0

In [235]:
X = pd.concat([X, tmp], axis=0)

In [236]:
del X['block_y']
del X['block_x']

In [237]:
X.index=range(len(X))

In [238]:
X.shape

(165564, 29)

In [239]:
X['f1'] = (X['x_x'] - X['x_y'])**2 + (X['y_x'] - X['y_y'])**2 + (X['z_x'] - X['z_y'])**2

In [240]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=0)
folds = []
fold_id = 0
X['fold'] = np.nan
for itr, ite in kf.split(X, X):
    X.ix[ite, 'fold'] = fold_id
    folds += [[itr, ite]]
    fold_id += 1

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


In [199]:
X

,x_x,y_x,z_x,volume_id_x,layer_id_x,module_id_x,dist_x,rad_dist_x,x_y,y_y,z_y,volume_id_y,layer_id_y,module_id_y,dist_y,rad_dist_y,target,fold
0,13.244800,-32.379101,598.000000,9,2,36,3.588278e+05,1223.830885,13.202800,-32.397400,597.500000,9.0,2.0,31.0,3.582302e+05,1223.905443,1,0.0
1,13.202800,-32.397400,597.500000,9,2,31,3.582302e+05,1223.905443,13.427000,-32.480999,602.500000,9.0,2.0,33.0,3.642415e+05,1235.299626,1,2.0
2,13.427000,-32.480999,602.500000,9,2,33,3.642415e+05,1235.299626,19.446199,-33.790699,698.000000,9.0,4.0,36.0,4.887240e+05,1519.966011,1,4.0
3,19.446199,-33.790699,698.000000,9,4,36,4.887240e+05,1519.966011,19.825199,-33.875198,702.000000,9.0,4.0,39.0,4.943446e+05,1540.567585,1,1.0
4,19.825199,-33.875198,702.000000,9,4,39,4.943446e+05,1540.567585,19.887100,-33.884499,702.500000,9.0,4.0,33.0,4.950499e+05,1543.656000,1,3.0
5,19.887100,-33.884499,702.500000,9,4,33,4.950499e+05,1543.656000,34.807499,-38.112099,817.500000,9.0,6.0,42.0,6.709703e+05,2664.094049,1,3.0
6,34.807499,-38.112099,817.500000,9,6,42,6.709703e+05,2664.094049,35.214199,-38.189499,822.000000,9.0,6.0,39.0,6.783825e+05,2698.477642,1,1.0
7,35.214199,-38.189499,822.000000,9,6,39,6.783825e+05,2698.477642,43.993198,-40.631302,957.500000,9.0,8.0,42.0,9.203926e+05,3586.304197,1,2.0
8,43.993198,-40.631302,957.500000,9,8,42,9.203926e+05,3586.304197,44.250301,-40.661499,962.000000,9.0,8.0,39.0,9.290554e+05,3611.446673,1,2.0
9,44.250301,-40.661499,962.000000,9,8,39,9.290554e+05,3611.446673,49.269100,-40.863300,1097.500000,9.0,10.0,42.0,1.208604e+06,4097.253547,1,2.0


In [243]:
X.columns

Index(['x_x', 'y_x', 'z_x', 'volume_id_x', 'layer_id_x', 'module_id_x',
       'dist_x', 'rad_dist_x', 'value_mean_x', 'value_max_x', 'value_min_x',
       'value_sum_x', 'value_std_x', 'value_count_x', 'x_y', 'y_y', 'z_y',
       'volume_id_y', 'layer_id_y', 'module_id_y', 'dist_y', 'rad_dist_y',
       'value_mean_y', 'value_max_y', 'value_min_y', 'value_sum_y',
       'value_std_y', 'value_count_y', 'target', 'f1', 'fold'],
      dtype='object')

In [241]:
#%%time
import lightgbm as lgbm
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

scores = []
for fold_id in range(5):
    lgb = lgbm.sklearn.LGBMClassifier()
    lgb.fit(X[X.fold != fold_id].drop(['fold', 'target'],axis=1), X[X.fold != fold_id]['target'])

    pred = lgb.predict(X[X.fold == fold_id].drop(['fold', 'target'],axis=1))#[:,1]
    scores += [accuracy_score( X[X.fold == fold_id]['target'], pred)]
scores = np.array(scores)
print(scores.mean(), scores.std())

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

0.9946606735294397 0.00020432967623064635


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [ ]:
0.9974847311877941 6.324812597346277e-05
0.994532583391577 0.00014189783980856174

In [246]:
imp = pd.DataFrame()
imp['feature'] = X.drop(['target', 'fold'], axis=1).columns.values
imp['importance'] = lgb.feature_importances_
imp = imp.sort_values('importance', ascending=False)
imp.index = range(len(imp))

In [247]:
imp

,feature,importance
0,f1,846
1,rad_dist_y,527
2,rad_dist_x,411
3,dist_y,263
4,dist_x,199
5,value_sum_x,118
6,value_sum_y,76
7,module_id_y,75
8,module_id_x,67
9,z_y,51


In [204]:
1-X['target'].mean()

0.5

In [149]:
X

,x_x,y_x,z_x,volume_id_x,layer_id_x,module_id_x,dist_x,rad_dist_x,x_y,y_y,z_y,volume_id_y,layer_id_y,module_id_y,dist_y,rad_dist_y,target,fold,f1
0,-38.421902,60.531502,100.994003,8,4,283,1.534009e+04,5.140305e+03,-61.762501,97.524200,165.076996,8.0,6.0,512.0,4.057599e+04,1.332558e+04,1,1.0,6019.873047
1,-61.762501,97.524200,165.076996,8,6,512,4.057599e+04,1.332558e+04,-91.977898,145.654999,248.330994,8.0,8.0,846.0,9.134359e+04,2.967531e+04,1,2.0,10160.771484
2,-91.977898,145.654999,248.330994,8,8,846,9.134359e+04,2.967531e+04,-139.442993,221.778000,379.600006,13.0,2.0,594.0,2.127260e+05,6.862983e+04,1,0.0,25279.199219
3,-139.442993,221.778000,379.600006,13,2,594,2.127260e+05,6.862983e+04,-188.819000,301.776001,517.400024,13.0,4.0,888.0,3.944242e+05,1.267214e+05,1,4.0,27826.515625
4,-188.819000,301.776001,517.400024,13,4,888,3.944242e+05,1.267214e+05,-191.951004,306.898010,526.400024,13.0,4.0,887.0,4.081286e+05,1.310316e+05,1,0.0,117.044426
5,-191.951004,306.898010,526.400024,13,4,887,4.081286e+05,1.310316e+05,-262.704987,422.963989,725.799988,13.0,6.0,1392.0,7.746981e+05,2.479124e+05,1,2.0,58237.781250
6,-262.704987,422.963989,725.799988,13,6,1392,7.746981e+05,2.479124e+05,-345.194000,560.494019,960.599976,13.0,8.0,2024.0,1.356065e+06,4.333124e+05,1,1.0,80849.984375
7,-345.194000,560.494019,960.599976,13,8,2024,1.356065e+06,4.333124e+05,-431.626007,707.424011,1217.500000,18.0,2.0,83.0,2.169056e+06,6.867497e+05,1,2.0,95056.539062
8,-431.626007,707.424011,1217.500000,18,2,83,2.169056e+06,6.867497e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0,NaN
9,-23.600700,21.074499,28.903299,8,2,126,1.836528e+03,1.001128e+03,-24.391100,21.806400,30.028799,8.0,2.0,127.0,1.972174e+03,1.070445e+03,1,0.0,2.427160
